<a href="https://colab.research.google.com/github/s1300200/GT/blob/main/%E5%A4%A7%E8%A6%8F%E6%A8%A1%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB%E5%85%A5%E9%96%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第一章

transformerのインストール

In [ ]:
!pip install transformers[ja,sentencepiece,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 39.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 697.9/697.9 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.0 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=99b34f1ea805a518876469fac5eac91c4552e776ccb6ffc2d586da5e06ff1de0
  Stored in directory: /root/.cache/pip/wheels/93/8b/55/dd5978a069678c372520847cf84ba2ec539cb41917c00a2206
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7403 sha256=c31f050b4622e0851aeb644d2606d5f95d6a3a8ebcb7237e13dad63b3

transformerにはモデルを簡単に扱うためのpipelinesという機能がある。
これをインポートしておく

In [ ]:
from transformers import pipeline

## 文書分類


テキストをあらかじめ定められたラベルに分類するタスク。スパムメールなどの自動判定にも使える。
また、テキストから読み取れる感情を検出する文書分類を**感情分析**と呼ぶ。

In [ ]:
text_classification_pipeline = pipeline(model = "llm-book/bert-base-japanese-v3-marc_ja")

Device set to use cpu


この関数をもとに、感情分析をしてみようじゃないか

In [ ]:
#textに自由に文章を埋め込む
text = "私には最高の彼女がおり、最高の友達たちがおり、最高の環境にいる"
text_classification_pipeline(text)

[{'label': 'positive', 'score': 0.9992996454238892}]

## 自然言語推論(NLI)

二つのテキストの論理関係を予測するタスク。言語モデルの意味理解能力を評価するために使用される。

In [ ]:
nli_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jnli")

Device set to use cpu


In [ ]:
#テキストを自由に入れる
text = "二人の男性がジェット機を見ています"
entailment_text = "ジェット機を見ている人が二人います"

print(nli_pipeline({"text" : text, "text_pair" : entailment_text}))

{'label': 'entailment', 'score': 0.9964311122894287}


## 意味的類似度計算(STS)

二つのテキストの意味が似ている度合いをスコアとして予測するタスク。情報検索や複数テキストの内容の性合成を確認する際に役立つ。

In [ ]:
text_sim_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-jsts",
    function_to_apply = "none",
)

config.json:   0%|          | 0.00/796 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
text = "私は最近、後輩たちの大会の応援に行きました"
text2 = "先週、後輩たちが出場する大会の応援を行いました"

result = text_sim_pipeline({"text" : text, "text_pair" : text2})
print(result["score"])

3.8882763385772705


### テキストの意味をベクトルで表現する分埋め込みモデルを用いたSTS


In [ ]:
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model = "llm-book/bert-base-japanese-v3-unsup-simcse-jawiki",
    task = "feature-extraction",
)


config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb =  sim_enc_pipeline(text2, return_tensors=True)[0][0]

sim_pair_score = cosine_similarity(text_emb, sim_emb, dim = 0)
print(sim_pair_score.item())

0.85164475440979


## 固有表現認識(NER)

テキストに含まれる固有表現を抽出するタスク。テキストデータから必要な情報を抽出するための基本的なタスク。

In [ ]:
from pprint import pprint

ner_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-ner-wikipedia-dataset",
    aggregation_strategy="simple",
)


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
text = "私は新世界の頂点に立つ。具体的には日本の首相。日本にとどまっているようじゃだめか、日本はね、出ていかないと"

pprint(ner_pipeline(text))


[{'end': None,
  'entity_group': '地名',
  'score': np.float32(0.9984693),
  'start': None,
  'word': '日本'},
 {'end': None,
  'entity_group': '地名',
  'score': np.float32(0.99835145),
  'start': None,
  'word': '日本'},
 {'end': None,
  'entity_group': '地名',
  'score': np.float32(0.9979944),
  'start': None,
  'word': '日本'}]


## 要約生成

暇だったらやる

## 急に溢れ出てきた語彙一覧

### 転移学習
ある、解きたいタスクに対して別の方法で学習したモデルを転用する方式。
下流タスク、事前学習が異なること
### 自己教師あり学習
モデルが自動的にラベルを生成して学習を行う方式。従来の教師あり学習は人手でラベルを付与してきたので、作成コストが制約となり大規模学習ができなかったが、それの解決策として編み出された。


## 大規模言語モデル(LLM)